In [1]:
import numpy as np
import h5py
import scipy.stats

In [2]:
# Loading the data
with h5py.File('../HAR/preprocessed.hdf5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    y_train = np.array(hf.get('y_train'))
    s_train = np.array(hf.get('s_train'))
    x_test = np.array(hf.get('x_test'))
    y_test = np.array(hf.get('y_test'))
    s_test = np.array(hf.get('s_test'))
    x_train_with_past = np.array(hf.get('x_train_with_past'))
    y_train_with_past = np.array(hf.get('y_train_with_past'))
    x_test_with_past = np.array(hf.get('x_test_with_past'))
    y_test_with_past = np.array(hf.get('y_test_with_past'))

# Infering HMM parameters using MLE

In [70]:
# Learning a one component Gaussian over all the features
def compute_transition(y, alpha=0.1):
    '''
    Compute the transition matrice.
    Rows: states to
    cols: states from
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    transition = alpha*np.ones((num_state, num_state))
    for i in xrange(y.shape[0]-1):
        transition[y[i+1]-1, y[i]-1] += 1
    # Normalisation (column should be normalized)
    transition /= np.sum(transition, axis=1)[:, np.newaxis]
    return transition

def compute_emission(x, y):
    '''
    Compute the parameters of the gaussian distribution
    of the emission given each state.
    We assume each emission distribution is independent,
    the covariance matrix is diagonal then.
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    
    sigma_diag = np.zeros((num_state, x.shape[1]))
    mu = np.zeros((num_state, x.shape[1]))
    for s in xrange(num_state):
        x_s = x[(y == s+1), :]
        # Computing mu_s
        mu[s] = np.mean(x_s, axis=0)
        # Computing sigma_s (by column)
        sigma_diag[s] = np.std(x_s, axis=0)

    return mu, sigma_diag

def compute_logscore(data, log_transition, mu, sigma_diag, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = compute_logB(data, mu, sigma_diag, j)

    return y + log_transition

def compute_logscore_pymc(data, log_transition, means, cov, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = scipy.stats.multivariate_normal.logpdf(data, mean=means[j], cov=covs[j])
    return y + log_transition

def viterbi(inputs, init, log_transition, mu, sigma_diag, C):
    '''
    Evaluates the highest scoring sequence.
    args: 
        inputs: observation
        init: initial probability distribution of the hidden states (C vector)
    '''
    y = np.zeros((C, C))
    initial = np.log(init)

    n = inputs.shape[0]
    # To store the maxes
    max_table = np.zeros((n, C))
    backpointer_table = np.zeros((n, C))

    # first timestep
    # the initial most likely paths are the initial state distribution
    state_init = initial + compute_logscore(inputs[0,:], log_transition, mu, sigma_diag, C)
    maxes = np.max(state_init, axis=1)
    backpointers = np.argmax(state_init, axis=1)
    max_table[0, :] = maxes

    for i in xrange(1, n):
        # COmputing the score
        y = compute_logscore(inputs[i, :], log_transition, mu, sigma_diag, C)
        scores = y + np.repeat(maxes.reshape(1, C), C, axis=0)

        # compute new maxes
        maxes = np.max(scores, axis=1)
        backpointers = np.argmax(scores, axis=1)

        max_table[i, :] = maxes
        backpointer_table[i, :] = backpointers

    # follow backpointers to recover max path
    classes = np.zeros(n)
    classes[n-1] = np.argmax(maxes, axis=0)
    for i in xrange(n-1, 0, -1):
        classes[i-1] = backpointer_table[i, classes[i]]

    return classes

def standardize(x):
    '''
    Standardize each column of x
    '''
    x_std = np.std(x, axis=0)
    x_mu = np.mean(x, axis=0)
    
    return (x - x_mu)/x_std[np.newaxis, :]

def compute_accuracy(pred_classes, true_classes):
    '''
    Compute accuracy
    '''
    return np.sum(pred_classes == true_classes) /(1.*len(pred_classes))

def compute_logB(data_point, mu, sigma_diag, j):
    '''
    Compute log(p(x|s_j))
    '''
    return np.sum([scipy.stats.norm.logpdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

def compute_B(data_point, mu, sigma_diag, j):
    '''
    Compute p(x|s_j)
    '''
    return np.prod([scipy.stats.norm.pdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

In [81]:
# We retain 6 features (known to be independent)

features = [0, 1, 2, 40, 41, 42, 80, 81, 82, 120, 121, 122] + range(555, 561)
x_sub_train = x_train[:, features]
x_sub_test = x_test[:, features]
print(x_sub_train.shape)

(7352, 18)


In [82]:
# Learning the HMM

# standardization
x_standard = standardize(x_sub_train)
x_sub_test_standard = standardize(x_sub_test)
print(x_standard.shape)

# ### TRANSITION
transition_train = compute_transition(y_train)
log_transition_train = np.log(transition_train)
print(transition_train.shape)

# ### EMISSION
mu, sigma_diag = compute_emission(x_standard, y_train)
print(mu.shape)
print(sigma_diag.shape)

(7352, 18)
(6, 6)
(6, 18)
(6, 18)


# Sequence Prediction

In [83]:
%%time
# Sequence prediction
C = 6
sample_size = x_standard.shape[0]
# uniform distribution for the inital state
initial = 1./C * np.ones(C)
seq_pred = viterbi(x_standard[:sample_size,:], initial, log_transition_train, mu, sigma_diag, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

ACCURACY train: 0.82154515778
CPU times: user 54.6 s, sys: 649 ms, total: 55.2 s
Wall time: 59.2 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [76]:
%%time
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], initial, log_transition_train, mu, sigma_diag, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

ACCURACY test: 0.768917543264
CPU times: user 6.79 s, sys: 27.8 ms, total: 6.82 s
Wall time: 6.91 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### More features

In [77]:
# We retain independent features from the train set, manually chosen
features = [0, 1, 2, 40, 41, 42, 80, 81, 82, 120, 121, 122] + range(555, 561)
x_sub_train = x_train[:, features]
x_sub_test = x_test[:, features]
print(x_sub_train.shape)

(7352, 18)


In [78]:
# Learning the HMM

# standardization
x_standard = standardize(x_sub_train)
print(x_standard.shape)

# ### TRANSITION
transition_train = compute_transition(y_train)
log_transition_train = np.log(transition_train)
print(transition_train.shape)

# ### EMISSION
mu, sigma_diag = compute_emission(x_standard, y_train)
print(mu.shape)
print(sigma_diag.shape)

(7352, 18)
(6, 6)
(6, 18)
(6, 18)


In [79]:
%%time
# Sequence prediction
C = 6
sample_size = x_standard.shape[0]
# uniform distribution for the inital state
initial = 1./C * np.ones(C)
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

ACCURACY train: 0.82154515778
CPU times: user 49.1 s, sys: 258 ms, total: 49.4 s
Wall time: 50.9 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [80]:
%%time
x_sub_test_standard = standardize(x_sub_test)
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

ACCURACY test: 0.771971496437
CPU times: user 19.4 s, sys: 74.5 ms, total: 19.4 s
Wall time: 19.9 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# Forward Backward algorithm

We implemented the forward backward algorithm but when fitted on the test set the results were lower than the HMM inferred from the train data set.

In [59]:
def forward(x, init, end, log_transition, mu, sigma_diag):
    '''
    args:
        x: observation sequence
        init: hidden state of the initialisation (assumed known)
        end: hidden state of the termination (assumed known)
        log_transition: log probability of the transition
        mu, sigma_diag: parameters for the 1d gaussian distribution
                        of the continous emission
    
    Compute log(p(X|lambda)) with lambda the HMM parameters
    (here transition, mu, sigma_diag)
    alpha[t, j] = p(x_1, ... x_t, S(t) = s_j|lambda)
    NB:
        alpha[0, :] used only as initialization
        log-sum-exp trick used
    '''
    C = mu.shape[0]
    T = x.shape[0]
    
    # Initialization
    alpha = np.zeros((T, C))
    alpha[0, init] = 1
    alpha[0,:] = np.log(alpha[0,:])
    
    # Recursion
    for t in xrange(1, T):
        
        for j in xrange(C):
            # Compute log p(S(t)|x_1, ..., x_{t-1})
            a = alpha[t-1, :] + log_transition[j,:]
            # Compute log evidence at time t in [1, T], x index shifted of 1
            b_t_j = compute_logB(x[t], mu, sigma_diag, j)
            a += b_t_j
            # log-sum-exp trick
            max_a = np.max(a)
            alpha[t, j] = max_a + np.log(np.sum(np.exp(a - max_a)))
        # Normalization
        Z_t = np.sum(np.exp(alpha[t, :]))
        alpha[t,:] -= np.log(Z_t) 
    
    return alpha

def backward(x, init, end, log_transition, mu, sigma_diag):
    '''
    Compute the log backward probabilities.
    beta[t, j] = p(x_{t+1}, ..., x_{T}| S(t) = s_j, lambda)
    '''
    C = mu.shape[0]
    T = x.shape[0]
    # Initialization
    beta = np.zeros((T, C))
    for i in xrange(C):
        #beta[T-1, i] = log_transition[end, i]
        beta[T-1, i] = 1./C
    
    # Recursion
    for t in xrange(T-2, -1, -1):
        for j in xrange(C):
            a = beta[t+1, :] + log_transition[:,j]
            for i in xrange(C):
                b_t_i = compute_logB(x[t+1], mu, sigma_diag, i)
                a[i] += b_t_i
            max_a = np.max(a)
            beta[t, j] = max_a + np.log(np.sum(np.exp(a - max_a))) 
        
    return beta

def compute_state_probability(alpha, beta):
    '''
    compute state occupation log probability
    '''
    # Removing the initialization
    gamma = alpha + beta
    # Normalization with log-sum-exp trick
    for t in xrange(gamma.shape[0]):
        row_max = np.max(gamma[t, :])
        Z_row = row_max + np.log(np.sum(np.exp(gamma[t, :] - row_max)))
        gamma[t, :] -= Z_row
    return gamma

def compute_state_transition(x, alpha, beta, log_transition, mu, sigma_diag):
    '''
    Compute log(p(S(t) = s_i, S(t+1) = s_j| X, lambda))
    output: psi of dim (T-1)*C*C, 2nd coord: i, 3nd coord: j
    '''
    T = x.shape[0]
    C = mu.shape[0]
    psi = np.zeros((T-1, C, C))
    for t in xrange(T-1):
        for j in xrange(C):
            b_t_j = compute_logB(x[t+1], mu, sigma_diag, j)
            for i in xrange(C):
                psi[t, i, j] = alpha[t, i] + log_transition[j, i] + beta[t+1, j] + b_t_j
        # Normalization with log sum exp trick
        mat_max = np.max(psi[t, :, :])
        Z_mat = mat_max + np.log(np.sum(np.exp(psi[t, :, :] - mat_max)))
        psi[t, :, :] -= Z_mat
    
    return psi
    
    
def forward_backward(x, init, end, log_transition, mu, sigma_diag, n_iterations):
    '''
    EM algorithm to fit an HMM to a sequence of observation.
    Take as argnument an initial HMM and returns a finer one.
    '''
    T = x.shape[0]
    C = mu.shape[0]
    
    # Copying object
    mu = mu.copy()
    sigma_diag = sigma_diag.copy()
    log_transition = log_transition.copy()
    
    for it in xrange(n_iterations):
        # ##### E-step: estimate the state occupation probabilities (in LOG)
        log_alpha = forward(x, init, end, log_transition, mu, sigma_diag)
        log_beta = backward(x, init, end, log_transition, mu, sigma_diag)
        log_state_probability = compute_state_probability(log_alpha, log_beta)
        log_state_transition = compute_state_transition(x, log_alpha, log_beta, log_transition, mu, sigma_diag)

        # ##### M-step: re-estimate HMM parameers
        state_probability = np.exp(log_state_probability)
        
        # Mu
        denom = np.sum(state_probability, axis=0)[np.newaxis, :]
        mu = np.dot(state_probability.T, x)/denom
    
        # Sigma_diag (stands for the standard deviation)
        for j in xrange(C):
            diff = x - mu[j]
            numerator = 0
            for t in xrange(T):
                numerator += state_probability[t, j]*(diff[t, :])**2
            sigma_diag[:, j] = numerator
        sigma_diag /= denom
        # Standard deviation from the variance
        sigma_diag = np.sqrt(sigma_diag)
        
        # Transition 
        state_transition = np.exp(log_state_transition)
        for j in xrange(C):
            for i in xrange(C):
                log_transition[i, j] = np.log(np.sum(state_transition[:, i, j]))
            # Normalization
            log_transition[:,j] -= np.log(np.sum(np.exp(log_transition[:, j])))
    
    return log_transition, mu, sigma_diag
    

In [60]:
%%time
init = 4
end = 1
n_iterations = 1
log_transition_train_new, mu_new, sigma_diag_new = forward_backward(x_standard, init, end, log_transition_train, mu, sigma_diag, n_iterations)

CPU times: user 2min 34s, sys: 1.85 s, total: 2min 36s
Wall time: 2min 50s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in log


In [61]:
# Comparing value
print mu
print mu_new

[[ 0.02522293 -0.00177132  0.0045369  -0.56560315 -0.45669749 -0.16650716]
 [-0.1787534  -0.21936092 -0.19922794 -0.80040918 -0.69653595  0.23307432]
 [ 0.1947258   0.03247523  0.05793074 -0.47849408 -0.42016397  0.20147609]
 [-0.01479045  0.13607265  0.04520951  0.30031038  0.1961177  -0.18645536]
 [ 0.06840119  0.03853049  0.03198569 -0.5189657  -0.34882962 -0.30448021]
 [-0.07539678 -0.01592395  0.03482719  1.67087662  1.38497542  0.29390974]]
[[  3.20748497e-02   5.11016732e-03   1.80171387e-02  -7.20131386e-01
   -3.24760551e-01  -1.46342944e-01]
 [ -1.02742925e-01  -1.46377109e-01  -1.29941861e-01  -1.05855769e+00
   -6.09759127e-01   2.30558421e-01]
 [  1.19262722e-01  -1.58474141e-02  -8.63224139e-03  -4.54759432e-01
   -1.62878405e-01   9.94591678e-02]
 [  2.12879330e-02   5.27697909e-02  -9.83257621e-03   4.22278167e-01
    1.87858896e-01  -1.38103410e-01]
 [  6.30245742e-02   2.40822329e-02  -4.11865495e-04  -8.12583731e-01
   -3.07952790e-01  -4.30835170e-01]
 [ -1.32907154

In [62]:
print sigma_diag
print sigma_diag_new

[[ 0.71640716  0.51146808  0.57251528  0.2204994   0.46464535  0.16410181]
 [ 1.11010689  0.90720976  1.06258925  0.2658547   0.57818387  0.51386202]
 [ 1.35294292  0.66268894  0.89404002  0.24217263  0.4707116   0.37867326]
 [ 0.59754132  0.79416585  0.80001113  0.5011433   0.6398332   0.79073217]
 [ 0.28594946  0.43716819  0.6298047   0.27047212  0.46420367  0.3867504 ]
 [ 1.44478089  1.80077441  1.5841213   0.84965169  1.13787962  1.96969504]]
[[ 0.71588287  1.18695089  1.28687945  0.36375394  0.11872733  1.53986457]
 [ 0.49937984  0.90743911  0.69755189  0.50722723  0.27305199  1.91454468]
 [ 0.55933534  1.12130238  0.949279    0.58216285  0.37517684  1.69698245]
 [ 0.23399855  0.28648606  0.19847864  0.44642535  0.47212612  1.17082647]
 [ 0.4687372   0.71436897  0.41550837  0.6507232   0.45286168  0.9379705 ]
 [ 0.1478589   0.56695283  0.29891633  0.43825587  0.17500975  2.05144979]]


In [63]:
print log_transition_train
print log_transition_train_new

[[-0.03526312 -9.41458649 -9.41458649 -9.41458649 -9.41458649 -3.37195365]
 [-9.28135786 -0.0520976  -2.98793858 -9.28135786 -9.28135786 -9.28135786]
 [-3.15421695 -4.24808989 -0.05897258 -9.19684978 -9.19684978 -9.19684978]
 [-9.46234345 -9.46234345 -9.46234345 -0.03439482 -3.41971062 -7.06444818]
 [-9.5277754  -3.55906784 -6.48325296 -9.5277754  -0.03067839 -9.5277754 ]
 [-9.5522265  -9.5522265  -9.5522265  -3.48611841 -9.5522265  -0.03139126]]
[[ -0.041809    -7.80833221  -3.00067162 -12.09935587  -8.31564974
   -5.89637263]
 [ -5.87007252  -0.06268429  -4.59823386 -10.84370918  -3.44096132
   -6.161124  ]
 [ -5.48580603  -3.21735932  -0.0646534  -10.46995994  -6.31672206
   -7.16918734]
 [ -7.80561917  -6.46173349  -6.11880653  -0.07136785  -4.81457402
   -3.43292002]
 [ -9.33960255  -4.24460649  -8.48779733  -3.07435083  -0.0468914
   -4.32094815]
 [ -3.39669097  -5.42853082  -7.88220199  -3.78946942  -5.62296144
   -0.05256675]]


As a sanity check the values obtained from the EM algorithm after 1 iteration are similar to those obtained from the fitting on the train set.

In [64]:
%%time
# Sequence prediction after 1 iteration
C = 6
sample_size = 1000
initial = 1./C * np.ones(C)
seq_pred = viterbi(x_standard[:sample_size,:], initial, log_transition_train_new, mu_new, sigma_diag_new, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

ACCURACY train: 0.481
CPU times: user 2.46 s, sys: 17.1 ms, total: 2.48 s
Wall time: 2.52 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [65]:
%%time
# Sequence prediction after 1 iteration
x_sub_test_standard = standardize(x_sub_test)
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], initial, log_transition_train_new, mu_new, sigma_diag_new, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

ACCURACY test: 0.442
CPU times: user 2.54 s, sys: 29.2 ms, total: 2.57 s
Wall time: 2.68 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


We kept the results from the first approach using HMM, with an accuracy of 77% on the test set. 